In [1]:
import os
import pickle
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
import numpy as np
from model import *
from dataloader import *
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

config = yaml.safe_load(open('config.yaml', 'r'))

In [2]:
dataset_path = "data/loaded_dataset"
dataset = Dataset()
sentences = None
if os.path.exists(dataset_path):
    with open(dataset_path, "rb") as f:
        dataset = pickle.load(f)
else:
    dataset.load()
    with open(dataset_path, "wb") as f:
        pickle.dump(dataset, f)

In [3]:
myList = []
track = 0
for midi in dataset.encoded_data_list:
    myList += midi[track].tolist()

mySet = set(myList)
vocab_size = len(mySet)
print(np.array(myList).shape)

beat2idx = {beat: i for i, beat in enumerate(mySet)}

(23674,)


In [4]:
def __split_input_target(chunk):
    global vocab_size
    input_text = chunk[:-1]
    target_text = tf.one_hot(chunk[1:], vocab_size)
    return input_text, target_text

idxOfBeat = np.array([beat2idx[beat] for beat in myList])
print(len(idxOfBeat))
tfDataset = tf.data.Dataset.from_tensor_slices(idxOfBeat)

sequences = tfDataset.batch(256+1, drop_remainder=True)
# o o o o o
#  \ \ \ \ \
#   x x x x x
ds = sequences.map(__split_input_target)
ds = ds.shuffle(10000).batch(config['params']['batch_size'], drop_remainder=True)
print(ds)

23674
<BatchDataset shapes: ((16, 256), (16, 256, 268)), types: (tf.int64, tf.float32)>


In [ ]:
for d in ds:
    print(d)

In [5]:
model = LoFiLoopNet('1to1-dev', vocab_size)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (16, None, 256)           68608     
_________________________________________________________________
lstm (LSTM)                  (16, None, 512)           1574912   
_________________________________________________________________
lstm_1 (LSTM)                (16, None, 512)           2099200   
_________________________________________________________________
lstm_2 (LSTM)                (16, None, 512)           2099200   
_________________________________________________________________
lstm_3 (LSTM)                (16, None, 512)           2099200   
_________________________________________________________________
dense (Dense)                (16, None, 268)           137484    
Total params: 8,078,604
Trainable params: 8,078,604
Non-trainable params: 0
______________________________________________

In [6]:
model.train(ds)

❌ Wandb Disabled in config.
Epoch 1/10


: 

: 

In [ ]:
model.load_weights('models/model-1to1-dev-9923-0.1200-bigger.hdf5')

loaded_model = model.model
model.model.predict()

### Old

In [ ]:
sentences = dataset.getConcatenatedTracks()
beatdict = BeatDictionary(sentences)
beatdict.get_related_chords('56-0.5-0')

In [ ]:
x_train, y_train = dataset.getSingleTrackTrainingData(track=0)

x_train = np.array([beatdict.vectorize(beat.tolist()) for beat in x_train])

y_train = beatdict.beat2index(y_train)
y_train = to_categorical(y_train, num_classes=beatdict.getDictSize())

print(x_train.shape, y_train.shape)

In [ ]:
model = LoFiLoopNet("dev", x_train.shape[1:], beatdict.getDictSize())
model.summary()

# Training

In [ ]:
model.train(x_train, y_train)

In [ ]:
sequence = model.generate(dataset, beatdict)
new_midi = MidiData()
new_midi.decode(sequence, filename='workflow_test4.mid')

# Inference

In [ ]:
model.load_weights("models/model-best.h5")

In [ ]:
sequence = model.generate(dataset, beatdict, topn=1)
new_midi = MidiData()
new_midi.decode(sequence, filename='exp.mid')

In [ ]:
import matplotlib.pyplot as plt
import random
from tqdm import tqdm

def predictAndPlot(beat, topn=1):
    pred, pred_beat = model.predict(beatdict, beat)

    topn = np.argsort(pred, axis=1)[:,-topn:][0]
    topn = np.flip(topn, 0)

    print(topn)

    return topn

    window = 2000
    bound = [topn[0] - window, topn[0] + window]
    bound = np.clip(bound, 0, len(beatdict.vector_dict))

    print(bound)

    title = ''
    for index, candidate in enumerate(topn):
        target = beatdict.vector_dict[candidate]
        target_prob = round(float(pred[0, candidate]), 3)
        title += f'({index+1}) [{candidate}] {target} : {target_prob} \n'


    f, ax = plt.subplots(1)
    # ax.bar(range(bound[1]-bound[0]), pred[0, bound[0]:bound[1]])
    ax.plot(range(bound[1]-bound[0]), pred[0, bound[0]:bound[1]])
    ax.set_xlim(left=bound[0], right=bound[1])
    plt.title(f"{beat} -> {pred_beat} \n {title}")
    plt.show(f)

# pick = random.sample(beatdict.vector_dict, 1)[0]
# predictAndPlot(pick, topn=5)

top_candidate_for_each_beat = []
for beat in tqdm(beatdict.vector_dict):
    topn = predictAndPlot(beat)
    top_candidate_for_each_beat.append(topn[0])

plt.bar(range(len(beatdict.vector_dict)), top_candidate_for_each_beat)
plt.show()



In [ ]:
random.sample([10], 1)

In [ ]:
print(dataset.pitch_range_list)

In [ ]:
model = LoFiLoopNet()
model.train(x_train, y_train)
model.plot()

In [ ]:
model = LoFiLoopNet()
model.load_last_weight()

In [ ]:
model.generate(dataset)